In [1]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

In [2]:
# load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model = model.cuda()

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
import pandas as pd
import numpy as np

In [4]:
!wget https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSentences.csv
!wget https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSplit.csv

--2023-12-10 23:01:31--  https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSentences.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1290254 (1.2M) [text/plain]
Saving to: ‘datasetSentences.csv’

datasetSentences.cs 100%[===================>]   1.23M  --.-KB/s    in 0.07s   

2023-12-10 23:01:31 (18.5 MB/s) - ‘datasetSentences.csv’ saved [1290254/1290254]

--2023-12-10 23:01:34--  https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSplit.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubuserc

In [5]:
df = pd.read_csv('datasetSentences.csv')
labels = pd.read_csv('datasetSplit.csv')

In [6]:
df

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,Emerges as something rare ; an issue movie tha...
...,...,...
11850,11851,A real snooze .
11851,11852,No surprises .
11852,11853,We 've seen the hippie-turned-yuppie plot befo...
11853,11854,Her fans walked out muttering words like `` ho...


In [7]:
def inp_ids(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['input_ids']

def token_types(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['token_type_ids']

def attn_mask(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['attention_mask']

In [8]:
df['input_ids'] = df['sentence'].apply(inp_ids)
df['token_type_ids'] = df['sentence'].apply(token_types)
df['attention_mask'] = df['sentence'].apply(attn_mask)

In [9]:
df

,sentence_index,sentence,input_ids,token_type_ids,attention_mask
0,1,The Rock is destined to be the 21st Century 's...,"[[tensor(101), tensor(1996), tensor(2600), ten...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
1,2,The gorgeously elaborate continuation of `` Th...,"[[tensor(101), tensor(1996), tensor(9882), ten...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
2,3,Effective but too-tepid biopic,"[[tensor(101), tensor(4621), tensor(2021), ten...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
3,4,If you sometimes like to go to the movies to h...,"[[tensor(101), tensor(2065), tensor(2017), ten...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
4,5,Emerges as something rare ; an issue movie tha...,"[[tensor(101), tensor(19391), tensor(2004), te...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
...,...,...,...,...,...
11850,11851,A real snooze .,"[[tensor(101), tensor(1037), tensor(2613), ten...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
11851,11852,No surprises .,"[[tensor(101), tensor(2053), tensor(20096), te...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
11852,11853,We 've seen the hippie-turned-yuppie plot befo...,"[[tensor(101), tensor(2057), tensor(1005), ten...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
11853,11854,Her fans walked out muttering words like `` ho...,"[[tensor(101), tensor(2014), tensor(4599), ten...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."


In [10]:
def extract_embeddings(row):
    inputs = {
        'input_ids': row['input_ids'].cuda(),
        'token_type_ids': row['token_type_ids'].cuda(),
        'attention_mask': row['attention_mask'].cuda()
    }

    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        return embeddings.cpu().numpy()

In [11]:
df['embeddings'] = df[['input_ids','token_type_ids','attention_mask']].apply(extract_embeddings, axis=1)

In [12]:
# Find the maximum sequence length
max_length = max([tensor.shape[1] for tensor in df.embeddings])

In [13]:
def pad_tensor(tensor, max_length):
    padding_length = max_length - tensor.shape[1]
    if padding_length > 0:
        pad =(0, 0, 0, padding_length)
        tensor = torch.nn.functional.pad(tensor, pad, "constant", 0)
    return tensor


In [14]:
# Pad each tensor in the DataFrame
X = [pad_tensor(torch.tensor(tensor), max_length) for tensor in df.embeddings]

In [15]:
# Now you can stack them
X = torch.vstack(X)

In [16]:
X = [tensor.mean(dim=0) for tensor in X]

In [17]:
X = torch.vstack(X)

In [18]:
X.shape

torch.Size([11855, 768])

In [19]:
labels

,sentence_index,splitset_label
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2
...,...,...
11850,11851,1
11851,11852,1
11852,11853,1
11853,11854,1


In [20]:
y = labels['splitset_label'].values

In [21]:
labels['splitset_label'].value_counts()

splitset_label
1    8544
2    2210
3    1101
Name: count, dtype: int64

In [22]:
from imblearn.over_sampling import SMOTE

# Instantiate SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE to the data
X_resampled, y_resampled = smote.fit_resample(X, y)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [23]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [24]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier with 100 trees (you can adjust this parameter)
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [25]:
# Make predictions on the test set
predictions = clf.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Accuracy: 0.9163253364540667
Classification Report:
               precision    recall  f1-score   support

           1       0.87      0.90      0.88      1744
           2       0.91      0.89      0.90      1705
           3       0.97      0.96      0.97      1678

    accuracy                           0.92      5127
   macro avg       0.92      0.92      0.92      5127
weighted avg       0.92      0.92      0.92      5127

